<a href="https://colab.research.google.com/github/Jonaslbb/Movies-Text-analysis-Recommender-Machine-Learning/blob/main/Notebook/Movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movies = pd.read_csv("https://raw.githubusercontent.com/MSvante/AAU-SDS-2020/master/Module2/tmdb_5000_movies.csv")
credits = pd.read_csv("https://raw.githubusercontent.com/MSvante/AAU-SDS-2020/master/Module2/tmdb_5000_credits.csv")

In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [ ]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


For this part of the assignment we'll only need 3 columns.
The most important is overview, which contains a description of the plot for all the movies. This will be our source for NLP.
In SML we'll try to predict genres and in UML we will create topics and find similar movies based on their overview.

In [ ]:
df = movies[["original_title","genres","overview"]]

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


We download stopwords. Stopwords are very common words which often carry little meaning. We want to remove these to reduce noise, so the model can focus on words with more meaning for the understanding of the text.

In [ ]:
from gensim.parsing.porter import PorterStemmer
stemmer = PorterStemmer()

In [ ]:
df['overview']= df['overview'].apply(str)
df["overview"]= [x.lower() for x in df.overview]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df["tokenized_text"] = df['overview'].map(lambda row: [tok.lower() for tok in word_tokenize(row) if tok not in stop_words and tok.isalpha()])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df["tokenized_text"] = df["tokenized_text"].map(lambda row: [stem for stem in stemmer.stem_documents(row)])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


We tokenize all the text. By tokenizing we split all words, which is necessary for some of the later code, like creating a dictionary.
We also remove stopwords and everything there isn't alphabetic letters.
Lastly we apply a stemmer on all text. This reduce every word to it's stem, so we dont have multiple word with the same meaning.

In [ ]:
# This is how "Pirates of the Caribbean: At World's End" looks after all the steps above
df.tokenized_text[1]

['captain',
 'barbossa',
 'long',
 'believ',
 'dead',
 'come',
 'back',
 'life',
 'head',
 'edg',
 'earth',
 'turner',
 'elizabeth',
 'swann',
 'noth',
 'quit',
 'seem']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
movies_train,movies_test = train_test_split(df,random_state=17)

We split our dataset into a test and training set, which will be necessary to make a SML model later. Since we didn't specify otherwise we'll get 75% of the data as training data, and the remaining as testing data

In [ ]:
from gensim.corpora.dictionary import Dictionary

In [ ]:
dictionary = Dictionary(movies_train['tokenized_text'])

In [ ]:
len(dictionary)

12817

In [ ]:
dictionary.filter_extremes(no_below = 20,no_above=0.1)

In [ ]:
len(dictionary)

1006

We create a dictionary with all words found in the overview texts. This gives 12817 different words. To remove even more noise we remove every word which occurs less than 20 times and every word which occurs in more than 10% of the overviews. We're left with 1006 words in our dictionary 

In [ ]:
import itertools
from collections import Counter

In [ ]:
words_movies = itertools.chain(*df['tokenized_text'])

In [ ]:
counter = Counter(words_movies)

In [ ]:
counter.most_common(10)

[('find', 846),
 ('life', 810),
 ('new', 744),
 ('on', 701),
 ('world', 643),
 ('young', 630),
 ('get', 607),
 ('friend', 573),
 ('live', 565),
 ('famili', 562)]

These are the 10 most common words in the overviews. All these words seems perfect to discribe the plot of many movies, so it makes sense these words are the most common

In [ ]:
train_bow = [dictionary.doc2bow(doc) for doc in movies_train['tokenized_text']]
test_bow = [dictionary.doc2bow(doc) for doc in movies_test['tokenized_text']]

In [ ]:
train_bow[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 2),
 (10, 1)]

In [ ]:
movies_train[:1]

,original_title,genres,overview,tokenized_text
1684,Walk the Line,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10402, ""n...",a chronicle of country music legend johnny cas...,"[chronicl, countri, music, legend, johnni, cas..."


In [ ]:
dictionary[1]

'countri'

We create a "bag of words" representation. A BoW contains the whole dictionary and makes docs by assigning 1 to words which occurs in the doc and 0 to everything else. The example above has 11 different words, and one of them occors 2 times. The reason the text only has 11 words is becouse we reduced the dictionary, so many of the words (eg. names) are flitered out.

In [ ]:
from gensim.models.tfidfmodel import TfidfModel
from gensim.models.lsimodel import LsiModel

In [ ]:
tfidf = TfidfModel(train_bow)

In [ ]:
train_tfidf = tfidf[train_bow]
test_tfidf = tfidf[test_bow]

In [ ]:
train_tfidf[0]

[(0, 0.2936569734425363),
 (1, 0.22310387911490048),
 (2, 0.1736616830066712),
 (3, 0.27154744124766245),
 (4, 0.2867548593789943),
 (5, 0.2936569734425363),
 (6, 0.288966451626396),
 (7, 0.28256734699642844),
 (8, 0.23092885066383262),
 (9, 0.5611614159177484),
 (10, 0.24506100488003563)]

In [ ]:
print(dictionary[2])
print(dictionary[9])

dai
record


Then we transform our data using tfidf. tfidf gives a numerical value based on how often a word is represented in the doc and dictionarty. If the word is super common it's given a low score and if it's uncommon it's given a high score. The reason is that specific/less used words often are important to the meaning of the text.
In the example above (walk the line) the word with the lowers score is "dai/day", while the word with the highest score is "record". It makes sense that "record" carries a lot of meaning when describing a movie about a musician.

In [ ]:
movies_lsi = LsiModel(train_tfidf, num_topics = 10, id2word=dictionary)

In [ ]:
train_corpus_lsi = movies_lsi[train_tfidf]

In [ ]:
movies_lsi.print_topics(10)

[(0,
  '0.133*"famili" + 0.124*"love" + 0.122*"stori" + 0.107*"year" + 0.100*"father" + 0.099*"must" + 0.096*"make" + 0.094*"film" + 0.091*"forc" + 0.089*"come"'),
 (1,
  '0.250*"love" + 0.208*"school" + -0.151*"war" + 0.151*"high" + -0.150*"forc" + 0.147*"stori" + -0.144*"agent" + -0.142*"mission" + -0.141*"team" + -0.140*"must"'),
 (2,
  '-0.536*"film" + -0.327*"stori" + -0.190*"war" + -0.185*"movi" + 0.185*"famili" + 0.164*"father" + 0.146*"son" + -0.143*"star" + 0.127*"mother" + 0.118*"daughter"'),
 (3,
  '-0.505*"school" + -0.376*"high" + 0.236*"stori" + 0.233*"famili" + -0.232*"student" + 0.148*"son" + -0.130*"teacher" + -0.129*"colleg" + 0.129*"mother" + 0.125*"father"'),
 (4,
  '0.367*"film" + -0.223*"love" + -0.215*"war" + -0.179*"stori" + 0.156*"murder" + 0.145*"polic" + 0.132*"york" + 0.129*"angel" + 0.126*"lo" + 0.120*"drug"'),
 (5,
  '-0.366*"love" + 0.273*"film" + -0.211*"fall" + 0.181*"town" + 0.165*"famili" + -0.157*"citi" + 0.149*"teenag" + -0.141*"york" + -0.135*"woma

We reduce dimensions using lsi on our tfidf data. The we print out the most common topics. It seems to be pretty generic movie topics like familiy, love, school and war. Nothing too exciting, so we'll make new dataframes containg "action","romance" and "Science fiction" to see how the topics differ.

In [ ]:
action = df[df["genres"].str.contains("Action")]
romance = df[df["genres"].str.contains("Romance")]
scifi = df[df["genres"].str.contains("Science Fiction")]

In [ ]:
words_action = itertools.chain(*action['tokenized_text'])
words_romance = itertools.chain(*romance['tokenized_text'])
words_scifi = itertools.chain(*scifi['tokenized_text'])

In [ ]:
counter_action = Counter(words_action)
counter_romance = Counter(words_romance)
counter_scifi = Counter(words_scifi)

In [ ]:
action_bow = [dictionary.doc2bow(doc) for doc in action['tokenized_text']]
romance_bow = [dictionary.doc2bow(doc) for doc in romance['tokenized_text']]
scifi_bow = [dictionary.doc2bow(doc) for doc in scifi['tokenized_text']]

In [ ]:
action_tfidf = tfidf[action_bow]
romance_tfidf = tfidf[romance_bow]
scifi_tfidf = tfidf[scifi_bow]

In [ ]:
action_lsi = LsiModel(action_tfidf, num_topics = 5, id2word=dictionary)
romance_lsi = LsiModel(romance_tfidf, num_topics = 5, id2word=dictionary)
scifi_lsi = LsiModel(scifi_tfidf, num_topics = 5, id2word=dictionary)

In [ ]:
counter_action.most_common(10)

[('find', 197),
 ('world', 186),
 ('on', 166),
 ('must', 165),
 ('new', 164),
 ('forc', 162),
 ('becom', 156),
 ('take', 143),
 ('get', 136),
 ('man', 134)]

In [ ]:
action_lsi.print_topics(5)

[(0,
  '0.156*"forc" + 0.153*"must" + 0.152*"agent" + 0.133*"team" + 0.124*"war" + 0.113*"mission" + 0.108*"fight" + 0.107*"power" + 0.107*"kill" + 0.105*"save"'),
 (1,
  '-0.328*"agent" + -0.229*"cop" + 0.182*"earth" + 0.179*"battl" + 0.175*"human" + 0.153*"planet" + 0.149*"race" + -0.145*"assassin" + 0.143*"vampir" + 0.140*"power"'),
 (2,
  '0.381*"agent" + -0.356*"cop" + 0.262*"bond" + 0.234*"mission" + -0.173*"polic" + 0.165*"jame" + -0.148*"citi" + 0.136*"cia" + -0.133*"gang" + -0.130*"crimin"'),
 (3,
  '0.496*"war" + 0.276*"battl" + 0.267*"soldier" + -0.224*"earth" + -0.181*"alien" + -0.171*"planet" + -0.147*"power" + -0.147*"save" + -0.128*"team" + 0.124*"armi"'),
 (4,
  '0.481*"assassin" + 0.270*"kill" + 0.232*"target" + -0.167*"bond" + -0.166*"cop" + -0.141*"must" + -0.139*"agent" + 0.137*"reveng" + -0.124*"citi" + 0.124*"vampir"')]

The topics of action movies are very diffent. This is pretty much what you use to explain the sterotypical action movie.

In [ ]:
counter_romance.most_common(10)

[('love', 288),
 ('life', 229),
 ('find', 178),
 ('young', 154),
 ('new', 141),
 ('get', 141),
 ('on', 137),
 ('friend', 131),
 ('man', 131),
 ('two', 127)]

In [ ]:
romance_lsi.print_topics(5)

[(0,
  '0.405*"love" + 0.225*"fall" + 0.163*"woman" + 0.138*"meet" + 0.127*"romanc" + 0.124*"girl" + 0.115*"relationship" + 0.114*"stori" + 0.107*"marri" + 0.095*"beauti"'),
 (1,
  '0.659*"love" + 0.417*"fall" + -0.140*"marriag" + -0.131*"marri" + -0.130*"relationship" + -0.095*"mother" + -0.091*"school" + -0.090*"coupl" + -0.090*"meet" + -0.089*"year"'),
 (2,
  '0.645*"romanc" + -0.231*"girl" + -0.198*"mother" + -0.175*"daughter" + -0.142*"town" + -0.137*"father" + 0.134*"relationship" + 0.121*"student" + 0.121*"stori" + -0.119*"small"'),
 (3,
  '0.339*"woman" + 0.284*"marri" + -0.274*"girl" + -0.267*"school" + 0.253*"marriag" + -0.234*"romanc" + -0.202*"high" + 0.194*"relationship" + 0.186*"coupl" + -0.144*"dai"'),
 (4,
  '0.418*"fall" + -0.303*"stori" + 0.226*"meet" + 0.209*"school" + 0.198*"high" + 0.182*"marri" + 0.176*"student" + 0.144*"colleg" + 0.135*"marriag" + -0.126*"husband"')]

Looks like romance movies are often about love, romance and marriage. Not a huge surprise.

In [ ]:
counter_scifi.most_common(10)

[('world', 128),
 ('find', 112),
 ('earth', 100),
 ('human', 93),
 ('on', 86),
 ('must', 82),
 ('planet', 81),
 ('year', 81),
 ('alien', 80),
 ('new', 80)]

In [ ]:
scifi_lsi.print_topics(5)

[(0,
  '0.330*"earth" + 0.316*"planet" + 0.292*"alien" + 0.222*"human" + 0.134*"space" + 0.134*"power" + 0.133*"race" + 0.127*"crew" + 0.123*"must" + 0.116*"year"'),
 (1,
  '-0.482*"planet" + -0.421*"alien" + -0.287*"earth" + 0.195*"futur" + 0.175*"robot" + -0.150*"crew" + 0.143*"power" + 0.129*"time" + 0.114*"evil" + 0.108*"citi"'),
 (2,
  '-0.566*"alien" + 0.323*"space" + 0.295*"planet" + -0.238*"human" + 0.216*"earth" + -0.166*"race" + 0.160*"crew" + 0.152*"futur" + 0.143*"mission" + -0.109*"small"'),
 (3,
  '-0.611*"human" + 0.324*"alien" + -0.276*"earth" + -0.178*"race" + 0.168*"time" + 0.105*"back" + 0.103*"secret" + -0.088*"power" + 0.085*"futur" + -0.085*"save"'),
 (4,
  '0.514*"earth" + -0.419*"planet" + -0.268*"robot" + -0.262*"human" + 0.205*"power" + 0.148*"stop" + 0.125*"govern" + -0.115*"futur" + 0.112*"space" + 0.102*"mission"')]

the main topics of Science fiction is earth, planet, human and aliens. Pretty much what you expect from the average science fiction movie. The model generally does well at finding topics for each genre.

#Supervised machinelearning

We'll try to make a SML model which can predict the genre of movies based on their overview

In [ ]:
from sklearn.linear_model import LogisticRegression
from gensim.matutils import corpus2csc,corpus2dense

In [ ]:
X_train = corpus2csc(train_bow)
X_test = corpus2csc(test_bow)

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train.T, movies_train["genres"])

In [ ]:
model.score(X_test.T, movies_test["genres"])

0.093255620316403

Our model is really bad and only gets a score if 0.09, but we're still going to defend it a bit. Most of the movies have multible genres and the model has to predict everyone correctly. So if our model guess 4/5 genres correctly it still counts as a wrong prediction. Also the genres are loosely defined and seems a bit arbitrary at times.

In [ ]:
predictions = pd.DataFrame(model.predict(X_test.T))
predictions = predictions.rename(columns={0:"predictions"})

In [ ]:
movies_test = movies_test.reset_index()

In [ ]:
new_df = movies_test.join(predictions)

In [ ]:
new_df[1:2]

,index,original_title,genres,overview,tokenized_text,predictions
1,108,Terminator Genisys,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...","the year is 2029. john connor, leader of the r...","[year, john, connor, leader, resist, continu, ...","[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam..."


In [ ]:
print(new_df.genres[1])
print(new_df.predictions[1])

[{'id': 878, 'name': 'Science Fiction'}, {'id': 28, 'name': 'Action'}, {'id': 53, 'name': 'Thriller'}, {'id': 12, 'name': 'Adventure'}]
[{'id': 28, 'name': 'Action'}, {'id': 53, 'name': 'Thriller'}, {'id': 878, 'name': 'Science Fiction'}]


This is a good example of why our model gets such a bad score. It predicts right with all three genres, but dosn't include the last genre (Adventure). We looked up the movie (Terminator Genisys) and it's classified as SciFi/Action/ Thriller/Adventure on most sites, but different sites have different combinations of the four and dosn't always include all four. So it's difficult for our model to predict genres when they are so losely applied.

In [ ]:
import json

In [ ]:
new_df["genres"] = [json.loads(row) for row in new_df["genres"]]
new_df["predictions"] =  [json.loads(row) for row in new_df["predictions"]]

In [ ]:
print(any(item in new_df.genres[0] for item in new_df.predictions[0]))
print(any(item in new_df.genres[1] for item in new_df.predictions[1]))
print(any(item in new_df.genres[2] for item in new_df.predictions[2]))
print(any(item in new_df.genres[3] for item in new_df.predictions[3]))
print(any(item in new_df.genres[4] for item in new_df.predictions[4]))
print(any(item in new_df.genres[5] for item in new_df.predictions[5]))
print(any(item in new_df.genres[6] for item in new_df.predictions[6]))
print(any(item in new_df.genres[7] for item in new_df.predictions[7]))
print(any(item in new_df.genres[8] for item in new_df.predictions[8]))
print(any(item in new_df.genres[9] for item in new_df.predictions[9]))
print(any(item in new_df.genres[10] for item in new_df.predictions[10]))
print(any(item in new_df.genres[11] for item in new_df.predictions[11]))
print(any(item in new_df.genres[12] for item in new_df.predictions[12]))
print(any(item in new_df.genres[13] for item in new_df.predictions[13]))
print(any(item in new_df.genres[14] for item in new_df.predictions[14]))
print(any(item in new_df.genres[15] for item in new_df.predictions[15]))
print(any(item in new_df.genres[16] for item in new_df.predictions[16]))
print(any(item in new_df.genres[17] for item in new_df.predictions[17]))
print(any(item in new_df.genres[18] for item in new_df.predictions[18]))
print(any(item in new_df.genres[19] for item in new_df.predictions[19]))

True
True
True
False
True
True
True
False
True
False
False
True
False
True
False
True
True
True
True
True


This shows true when the model predicts at least one genre correctly. in 14/20 cases it predicts one of the genres right. This just show that, while still not being great, the model is better than the score otherwise sugest.

We coudn't get that code to run the whole dataset, so we had to do it manually. Thats why we only included a sample size of 20.

# Recommender
Lastly we want to build a recommender that can recommend movies based on text or title.

In [ ]:
from gensim.similarities import MatrixSimilarity

In [ ]:
train_lsi_corpus = corpus2dense(train_corpus_lsi, 100 )

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:502: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = np.column_stack(sparse2full(doc, num_terms) for doc in corpus)


In [ ]:
document_topic_matrix_train = MatrixSimilarity(train_corpus_lsi)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
model2 = LogisticRegression()

model2.fit(train_lsi_corpus.T, movies_train.original_title)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
def recommender_text(text):
  vec_bow = dictionary.doc2bow(text.lower().split()) # convert to bag of words
  vec_tfidf = tfidf[vec_bow] # convert to tfidf
  vec_lsi = movies_lsi[vec_tfidf]  # convert to LSI
  sim = document_topic_matrix_train[vec_lsi] # # finding similar movies

  sim = sorted(enumerate(sim), key=lambda item: -item[1])
  for i, s in sim[:10]: #ten most similar texts
    print(s, movies_train.iloc[i,:]['original_title'])

In [ ]:
def recommender_title(movie):
  mov = df[df.original_title ==movie]
  doc = pd.DataFrame(mov["overview"]).to_string(index=False)
  vec_bow = dictionary.doc2bow(doc.lower().split()) # convert to bag of words
  vec_tfidf = tfidf[vec_bow] # convert to tfidf
  vec_lsi = movies_lsi[vec_tfidf]  # convert to LSI
  sim = document_topic_matrix_train[vec_lsi] # finding similar movies

  sim = sorted(enumerate(sim), key=lambda item: -item[1])
  for i, s in sim[:10]: #ten most similar texts
    print(s, movies_train.iloc[i,:]['original_title'])

We basically train a model to find similarities between topics. 

We define two recommender functions which recommends similar movies based on text or title. The one based on text takes text that your find interesting, and turns it into BoW -> tfidf -> LSI and then find similar movies.

The one based on movie title pretty much does the same. I just take a movie title and find the overview for that title, and uses that insted of free text.

In [ ]:
recommender_text("A movie about a man and a woman who finds eachother and falls in love and gets married")

0.9870193 Shallow Hal
0.9736098 Tom Jones
0.9585324 Boys Don't Cry
0.9510197 Mary Reilly
0.95028865 Swept Away
0.9490335 You've Got Mail
0.94870824 Goddess of Love
0.9483028 Two Girls and a Guy
0.9473694 Never Again
0.9471235 Entrapment


Wrote a pretty stereotypical description of a romantic movie and got recommended 10 romantic movies, so the model seems to work

In [ ]:
recommender_text("a spacecrew goes to another planet in a spaceship and finds aliens and mystery")

0.98720086 The Hitchhiker's Guide to the Galaxy
0.9672271 キャプテンハーロック
0.95243484 Lost in Space
0.948331 Rejsen til Saturn
0.94221497 Meet Dave
0.93761617 Space Battleship Yamato
0.93724334 Faith Connections
0.93302846 Flash Gordon
0.92076385 Jason X
0.9206224 Battlefield Earth


This time we tried with a SciFi text.

In [ ]:
recommender_text("a hero saves the city by fighting and killing evil criminal terrorist villians")

0.95327806 Foodfight!
0.94307065 Star Wars
0.917028 Iron Man
0.91555077 American Ninja 2: The Confrontation
0.90863323 Percy Jackson: Sea of Monsters
0.9077703 Tears of the Sun
0.90360516 A Bug's Life
0.9002557 Gods of Egypt
0.89945877 Yoga Hosers
0.8974423 Lara Croft Tomb Raider: The Cradle of Life


Lastly we tried with a action plot and got these 10 movies. The model doesn't necessarily recommend good movies, but this could be fixed easily by simply removing movies with a low score

In [ ]:
recommender_title("Walk the Line")

0.93166286 I'm Not There.
0.9119216 Dreamgirls
0.8929023 House of D
0.8835296 Bullets Over Broadway
0.87108594 Chasing Mavericks
0.86903495 The Broken Hearts Club: A Romantic Comedy
0.8657029 20 Feet from Stardom
0.85778624 Water
0.85426545 Ha-Buah
0.84962314 Green Room


In [ ]:
recommender_title("Avatar")

0.95500165 The Hitchhiker's Guide to the Galaxy
0.94261056 Escape from Planet Earth
0.93323934 Faith Connections
0.9321781 Titan A.E.
0.90950584 Meet Dave
0.8944231 Konferenz der Tiere
0.88850063 Star Trek IV: The Voyage Home
0.88645846 Rejsen til Saturn
0.8795259 キャプテンハーロック
0.87795913 Space Battleship Yamato


In [ ]:
recommender_title("Indiana Jones and the Last Crusade")

0.9772423 Maze Runner: The Scorch Trials
0.9760797 Capricorn One
0.9669175 Transcendence
0.9599792 Star Wars
0.95481193 Kung Fu Panda 2
0.95388263 Sanctum
0.9508109 G-Force
0.95037246 X2
0.9493724 兔侠传奇
0.94769317 Death Race 2000


It works with titles as well. The model only takes text into account, so it could definitely be improved by including other data as well. There are multiple Indiana Jones movies in the dataset, so a better recommender would probably recommend them before Kung fu panda 2, by taking actors, directors eg. into account.